<a href="https://colab.research.google.com/github/limkaram/Natural_language_processing_with_deep_learning/blob/main/Movies_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 코사인 유사도 활용 영화 추천 시스템

* The Movices Dataset 출처 : https://www.kaggle.com/rounakbanik/the-movies-dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/My Drive/Natural_language_processing_with_deep_learning/movies_metadata.csv')
nessesary_columns = ['original_title', 'overview']
data = data[nessesary_columns].head(20000)

data['overview'] = data['overview'].fillna('')
data['overview'].isnull().sum()

* tf-idf 진행시 데이터에 Null이 들어가 있으면, 에러 발생
* 사전 Null 값 제거

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# tf-idf 진행
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['overview'])

print(tfidf_matrix.shape)
print(type(tfidf_matrix.shape))

* tfidf_matrix.shape 결과 (20000, 47788) : 20000개의 영화를 표현하기 위해서 47788개의 단어가 사용됨을 보여줌

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
# 영화 타이틀 입력시 인덱스를 리턴하는 Table
indices = pd.Series(data.index, index=data['original_title']).drop_duplicates()

In [ ]:
def get_recommender(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옴
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옴
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목 리턴
    return data['original_title'].iloc[movie_indices]

get_recommender('The Dark Knight Rises')